In [1]:
from keras_segmentation.models.segnet import segnet
model = segnet(n_classes=2 ,  input_height=448, input_width=576)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

2024-04-08 22:15:45.129049: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


Error in loading augmentation, can't import imgaug.Please make sure it is installed.


2024-04-08 22:15:49.931333: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-04-08 22:15:49.934349: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-04-08 22:15:51.469828: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:927] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-08 22:15:51.469910: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3050 Laptop GPU computeCapability: 8.6
coreClock: 1.057GHz coreCount: 16 deviceMemorySize: 4.00GiB deviceMemoryBandwidth: 163.94GiB/s
2024-04-08 22:15:51.469951: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2024-04-08 22:15:51.489089: I tensorflow/stream_exec

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 448, 576, 3)]     0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 450, 578, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 448, 576, 64)      1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 448, 576, 64)      256       
_________________________________________________________________
activation (Activation)      (None, 448, 576, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 224, 288, 64)      0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 226, 290, 64)      0   

In [2]:
model.train(
    train_images =  "app/train_img",
    train_annotations = "app/train_ano",
    checkpoints_path = "app/checkpoints" , epochs= 2, 
    batch_size=2,
    steps_per_epoch=4006,
    validate=True,
    val_images="app/val_img",
    val_annotations="app/val_ano", 
    val_batch_size=2,
    val_steps_per_epoch=500,
)

Verifying training dataset
Verifying validation dataset


2024-04-08 22:16:53.343364: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-04-08 22:16:53.345653: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3293725000 Hz


Epoch 1/2


2024-04-08 22:16:54.178429: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2024-04-08 22:16:58.550115: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-04-08 22:16:58.550319: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Unimplemented: /usr/local/cuda-11.0/bin/ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-04-08 22:16:58.585235: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The pt

   1/4006 [..............................] - ETA: 33:07:35 - loss: 1.1464 - accuracy: 0.3775

2024-04-08 22:17:23.091484: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-04-08 22:17:23.125518: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'



4006/4006 [==============================] - 641s 152ms/step - loss: 0.3442 - accuracy: 0.8724 - val_loss: 0.3807 - val_accuracy: 0.8672

Epoch 00001: saving model to app/checkpoints.00001
Epoch 2/2
4006/4006 [==============================] - 600s 150ms/step - loss: 0.2622 - accuracy: 0.9016 - val_loss: 0.6730 - val_accuracy: 0.7363

Epoch 00002: saving model to app/checkpoints.00002


In [3]:
def mask_conversion(inp_dict,mask):
    keys_list = list(inp_dict.keys())
    for i in keys_list:
        mask[mask == i] = inp_dict[i]
    return mask
import os 
if not os.path.exists("app/testout"):
    os.mkdir("app/testout")


if not os.path.exists("app/testpre"):
    os.mkdir("app/testpre")

import cv2
z=cv2.imread("app/test_img/ISIC_0024311.jpg")
dic={1:255}
x = model.predict_segmentation(z)
image_decoded=mask_conversion(dic,x)
cv2.imencode('.png', image_decoded)[1].tofile("app/temp3.png")

import os 
for i , j in enumerate(os.listdir("app/test_img")):
    z=cv2.imread(os.path.join("app/test_img",j))
    v=cv2.imread(os.path.join("app/test_ano",j.replace(".jpg",".png")))
    dic={1:255}
    predicted_img = model.predict_segmentation(z)
    decoded_predicted_img=mask_conversion(dic,predicted_img)
    decoded_mask=mask_conversion(dic,v)
    # Resize the intersection mask using cv2
    resized_decoded_predicted_img = cv2.resize(decoded_predicted_img, (600, 450), interpolation=cv2.INTER_NEAREST)
    cv2.imencode('.png', resized_decoded_predicted_img)[1].tofile("app/testout/"+j.replace(".jpg","")+".png")
    cv2.imencode('.png', decoded_mask)[1].tofile("app/testpre/"+j.replace(".jpg","")+".png")


import numpy as np
def calculate_iou(mask1, mask2):
    """
    Calculate Intersection over Union (IoU) between two segmentation masks.

    Parameters:
        mask1 (numpy.ndarray): First segmentation mask.
        mask2 (numpy.ndarray): Second segmentation mask.
        
    Returns:
        float: IoU value.
    """
    
    intersection = np.logical_and(mask1, mask2)
    union = np.logical_or(mask1, mask2)
    iou = np.sum(intersection) / np.sum(union)
    return iou

lst=[]
for i, x in enumerate(os.listdir("app/test_ano")):
    y=os.path.join("app/testout",x)
    x=os.path.join("app/testpre",x)
    x=cv2.imread(x)
    y=cv2.imread(y)
    x=np.array(x,dtype=np.uint8)
    y=np.array(y,dtype=np.uint8)
    lst.append(calculate_iou(x,y))
print(sum(lst)/i)

2024-04-08 22:38:11.655063: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-04-08 22:38:11.698758: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-04-08 22:38:11.760660: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-04-08 22:38:11.809291: W tensorflow/stream_executor/gpu/

0.49698669430452036
